### Procedural Generation Using Semi-Random Color Palettes

In [ ]:
import json
import gzip
import random
import math
import datetime

# ==============================================================================
# 1. UTILITY & HELPER FUNCTIONS
# These are Python versions of the core utilities from the JavaScript code.
# ==============================================================================

def hsl_to_hex(h, s, l):
    """Converts HSL color to HEX format."""
    s /= 100
    l /= 100
    a = s * min(l, 1 - l)
    def f(n):
        k = (n + h / 30) % 12
        color = l - a * max(-1, min(k - 3, 9 - k, 1))
        return round(255 * color)
    return f'#{f(0):02x}{f(8):02x}{f(4):02x}'

# Seeded Random Number Generator (for deterministic results)
class Mulberry32:
    def __init__(self, seed):
        self.seed = seed

    def __call__(self):
        self.seed = (self.seed + 0x6D2B79F5) & 0xFFFFFFFF
        t = self.seed
        t = ((t ^ (t >> 15)) * 0x1B873593) & 0xFFFFFFFF
        t = t ^ (t >> 7)
        t = ((t ^ (t >> 7)) * 0x6C5565A5) & 0xFFFFFFFF
        t = t ^ (t >> 14)
        return (t ^ (t >> 14)) / 4294967296

# ==============================================================================
# 2. FRACTAL & OBJECT BUILDER FUNCTIONS
# Python versions of the algorithms from fractals.js and main.js.
# ==============================================================================

def build_mountain_range(start, end, detail, height, jaggedness):
    """Generates mountain ridgeline points using midpoint displacement."""
    points = [start, end]
    displacement = height
    for _ in range(detail):
        new_points = [points[0]]
        for i in range(len(points) - 1):
            p1 = points[i]
            p2 = points[i+1]
            mid_x = (p1['x'] + p2['x']) / 2
            mid_y = (p1['y'] + p2['y']) / 2
            offset = (random.random() - 0.5) * displacement
            new_points.append({'x': mid_x, 'y': mid_y + offset})
            new_points.append(p2)
        points = new_points
        displacement *= jaggedness
    return points

def build_tree_segments(params):
    """Recursively generates the segment data for a fractal tree."""
    segments = []
    rand = Mulberry32(params.get('rngSeed', int(random.random() * 1e9)))
    
    len_rand_map = [((rand() - 0.5) * params.get('lenRand', 0)) for _ in range(params['levels'])] if params.get('unifyLenPerLevel') else None
    angle_rand_map = [((rand() - 0.5) * params.get('angleRand', 0) * 0.15) for _ in range(params['levels'])] if params.get('unifyAnglePerLevel') else None
    uniform_jitter = (rand() - 0.5) * params.get('angleRand', 0) * 0.15 if params.get('uniformAngleRand') else None

    def branch(x, y, length, angle, depth, level, parent_idx):
        if depth <= 0 or length < 0.6:
            return None
        
        x2 = x + length * math.cos(angle)
        y2 = y - length * math.sin(angle)
        
        seg_idx = len(segments)
        segments.append({
            'level': level, 'len': length, 'baseAng': angle, 'parent': parent_idx, 'children': [],
            'x1': x, 'y1': y, 'x2': x2, 'y2': y2
        })

        len_randomness = len_rand_map[level] if (len_rand_map and level < len(len_rand_map)) else (rand() - 0.5) * params.get('lenRand', 0)
        new_length = length * (params.get('lenScale', 0.68) + len_randomness)

        if uniform_jitter is not None:
            jitter = uniform_jitter
        elif angle_rand_map and level < len(angle_rand_map):
            jitter = angle_rand_map[level]
        else:
            jitter = (rand() - 0.5) * params.get('angleRand', 0) * 0.15
        
        spread = params.get('angle', 25) * math.pi / 180

        left_idx = branch(x2, y2, new_length, angle - spread + jitter, depth - 1, level + 1, seg_idx)
        right_idx = branch(x2, y2, new_length, angle + spread + jitter, depth - 1, level + 1, seg_idx)
        
        if left_idx is not None: segments[seg_idx]['children'].append(left_idx)
        if right_idx is not None: segments[seg_idx]['children'].append(right_idx)
        
        return seg_idx

    branch(params['x'], params['y'], params['baseLen'], math.pi / 2, params['levels'], 0, -1)
    return segments

def build_flower_segments(params):
    """Generates flower segments using an L-System."""
    s = 'F'
    for _ in range(params['iter']):
        s = s.replace('F', 'F[+F]F[-F]F')
    
    segments, stack = [], []
    x, y, angle = params['cx'], params['cy'], -math.pi / 2
    step = params['step']

    for char in s:
        if char == 'F':
            nx, ny = x + step * math.cos(angle), y + step * math.sin(angle)
            segments.append({'x1': x, 'y1': y, 'x2': nx, 'y2': ny})
            x, y = nx, ny
        elif char == '+':
            angle += params['angle'] * math.pi / 180
        elif char == '-':
            angle -= params['angle'] * math.pi / 180
        elif char == '[':
            stack.append({'x': x, 'y': y, 'angle': angle})
        elif char == ']' and stack:
            state = stack.pop()
            x, y, angle = state['x'], state['y'], state['angle']
    return segments

def find_flower_tips(flower_data):
    """
    Analyzes flower segments to find endpoints that are not start points,
    and adds them to a 'tips' array in the flower data.
    """
    if not flower_data.get('segments'):
        flower_data['tips'] = []
        return

    start_points = set()
    end_points = {}

    for seg in flower_data['segments']:
        start_key = f"{seg['x1']},{seg['y1']}"
        end_key = f"{seg['x2']},{seg['y2']}"
        start_points.add(start_key)
        end_points[end_key] = {'x': seg['x2'], 'y': seg['y2']}

    tips = [point for key, point in end_points.items() if key not in start_points]
    flower_data['tips'] = tips

# ==============================================================================
# 3. SCENE GENERATION RECIPE
# This is where you define the procedure for creating your artwork.
# ==============================================================================

def generate_my_recipe(canvas_width=1920, canvas_height=1080):
    """
    Generates a scene based on the user's described recipe.
    Modify the parameters here to create different variations!
    """
    scene = []
    
    # --- 1. Background Settings ---
    bg1 = hsl_to_hex(random.randint(190, 240), 50, 10)
    bg2 = hsl_to_hex(random.randint(200, 250), 60, 40)
    gradient = True
    
    # --- 2. Palette Settings ---
    base_hue = random.randint(0, 360)
    palette = [hsl_to_hex(base_hue + i*15, random.randint(50, 80), random.randint(40, 90)) for i in range(10)]

    # --- 3. Sun/Moon Object ---
    sun_size = random.uniform(canvas_width / 9, canvas_width / 6)
    scene.append({
        'type': 'celestial',
        'data': {
            'cx': random.uniform(sun_size, canvas_width - sun_size),
            'cy': random.uniform(sun_size * 0.75, canvas_height * 0.4),
            'size': sun_size, 'glow': sun_size * random.uniform(0.6, 1.0),
            'color': random.choice(['#FFFFFF', '#FFD700', '#FFF4E6', '#FFC0CB']), 'alpha': 1.0
        }
    })
    
    # --- 4. Cloud Objects ---
    # for _ in range(random.randint(100, 200)):
    #     cloud_dmax = random.uniform(20, 100)
    #     cloud_puffs = [{
    #         'r': random.uniform(cloud_dmax * 0.2, cloud_dmax) / 2,
    #         'color': hsl_to_hex(random.randint(0, 360), random.randint(5, 20), random.randint(85, 98)),
    #         'offsetX': (random.random() - 0.5) * cloud_dmax,
    #         'offsetY': (random.random() - 0.5) * cloud_dmax * 0.5
    #     } for _ in range(random.randint(3, 5))]
    #     scene.append({
    #         'type': 'clouds', 'data': {
    #             'cx': random.uniform(0, canvas_width), 'cy': random.uniform(50, canvas_height * 0.5),
    #             'circles': cloud_puffs, 'blur': random.uniform(10, 20),
    #             'shadowColor': '#FFFFFF', 'shadowX': random.uniform(-10, 10), 'shadowY': random.uniform(-10, 10),
    #             'alpha': random.uniform(0.3, 0.6)
    #         }
    #     })
    
    # --- 4. Cloud Objects (Grouped into Bands) ---
    num_cloud_bands = random.randint(12, 24)
    for band_num in range(num_cloud_bands):
        band_y = random.uniform(50, canvas_height * 0.5)
        num_clouds_in_band = random.randint(10, 20)
        band_start_x = random.uniform(-canvas_width * 0.2, canvas_width * 0.8)
        
        for i in range(num_clouds_in_band):
            cloud_dmax = random.uniform(20, 100)
            cloud_puffs = [{
                'r': random.uniform(cloud_dmax * 0.2, cloud_dmax) / 2,
                'color': hsl_to_hex(random.randint(0, 360), random.randint(5, 20), random.randint(85, 98)),
                'offsetX': (random.random() - 0.5) * cloud_dmax,
                'offsetY': (random.random() - 0.5) * cloud_dmax * 0.5
            } for _ in range(random.randint(3, 5))]
            
            scene.append({
                'type': 'clouds', 'data': {
                    'cx': band_start_x + i * random.uniform(80, 120),
                    'cy': band_y + random.uniform(-10, 10),
                    'circles': cloud_puffs, 'blur': random.uniform(10, 20),
                    'shadowColor': '#FFFFFF', 'shadowX': random.uniform(-10, 10), 'shadowY': random.uniform(-10, 10),
                    'alpha': random.uniform(0.3, 0.5)
                }
            })

    # --- 5. Mountain Objects ---
    mountain_ridges = []
    num_mountains = 4
    for i in range(num_mountains):
        base_y_start = canvas_height * (0.4 + i * 0.12)
        base_y_end = canvas_height * (0.45 + i * 0.12)
        base_y = random.uniform(base_y_start, base_y_end)
        
        angle_rad = math.radians(random.uniform(-10, 10))
        y_offset = math.tan(angle_rad) * (canvas_width + 100) / 2
        
        mtn_start = {'x': -50, 'y': base_y - y_offset}
        mtn_end = {'x': canvas_width + 50, 'y': base_y + y_offset}
        mtn_data = {
            'start': mtn_start, 'end': mtn_end, 'detail': 8, 'height': 200 / (i + 1),
            'jaggedness': 0.6, 'isSmooth': i == num_mountains - 1,
            'color': hsl_to_hex(base_hue, random.randint(20, 40), 25 - i * 4), 'alpha': 1.0,
            'hasGradient': True, 'color2': random.choice(['#CCCCCC', '#FFD777', '#FFF4E6', '#FFC0CB']), 'alpha': 1.0
        }
        mtn_data['points'] = build_mountain_range(mtn_start, mtn_end, mtn_data['detail'], mtn_data['height'], mtn_data['jaggedness'])
        
        scene.append({'type': 'mountain', 'data': mtn_data})
        mountain_ridges.append(mtn_data['points'])

    # --- 6. Tree Objects ---
    num_trees = 0
    TOTAL_TREE_CAP = 100
    for i, ridge in enumerate(mountain_ridges):
        trees_on_this_ridge = random.randint(15 - i * 3, 30 - i * 5)
        # trees_on_this_ridge = random.randint(10, 20 + i * 5)
        for _ in range(trees_on_this_ridge):
            if num_trees >= TOTAL_TREE_CAP: break
            
            point_on_ridge = random.choice(ridge)
            
            tree_params = {
                'x': point_on_ridge['x'], 'y': point_on_ridge['y'],
                'levels': random.randint(6, 8),
                'baseLen': (40 + i * 15) * random.uniform(0.8, 1.2),
                'lenScale': random.uniform(0.60, 0.80),
                'angle': random.uniform(10, 25),
                'lenRand': 0.2, 'angleRand': 0.2,
                'baseWidth': (8 + i * 3) * random.uniform(0.8, 1.2),
                'widthScale': 0.7, 'branchColors': palette, 'levelAlphas': [1.0] * 10,
                'hasBlossoms': i >= num_mountains - 2 and random.random() > 0.5,
                'blossomColor': '#FFFFFF', 'blossomSize': 3,
                'rngSeed': int(random.random() * 1e9), 'randomColor': False, 'randomColorPerLevel': False,
                'uniformAngleRand': False, 'unifyLenPerLevel': False, 'unifyAnglePerLevel': False,
                'hasShadow': False, 'shadowColor': '#000000', 'shadowBlur': 0, 'shadowX': 0, 'shadowY': 0
            }
            tree_data = {**tree_params, 'segments': build_tree_segments(tree_params)}
            scene.append({'type': 'tree', 'data': tree_data})
            num_trees += 1
            
    # --- 7. (Optional) Ferns and Flowers on the front-most ridge ---
    front_ridge = mountain_ridges[-1]
    # Add Ferns
    for _ in range(random.randint(20, 50)):
        point = random.choice(front_ridge)
        y_offset = random.uniform(0, canvas_height * 0.15)
        scene.append({
            'type': 'fern', 'data': {
                'cx': point['x'], 'cy': point['y'] + y_offset,
                'size': (canvas_height * 0.01) * random.uniform(0.8, 1.5),
                'points': 1500, 'color': random.choice(palette), 'rngSeed': int(random.random() * 1e9),
                'alpha': 1.0, 'isSpaceFern': True
            }
        })
    # Add Flowers
    for _ in range(random.randint(15, 40)):
        point = random.choice(front_ridge)
        y_offset = random.uniform(0, canvas_height * 0.15)
        flower_params = {
            'cx': point['x'], 'cy': point['y'] + y_offset, 'iter': 3,
            'angle': random.uniform(15, 40), 'step': (canvas_height * 0.01) * random.uniform(0.5, 0.8),
            'stroke': 2.0, 'alpha': 1.0, 'color': random.choice(palette),
            'hasBlossoms': True, 'blossomSize': 2, 'blossomColor': hsl_to_hex(random.randint(0,360), 80, 90)
        }
        flower_data = {**flower_params, 'segments': build_flower_segments(flower_params)}
        find_flower_tips(flower_data) # <--- THIS IS THE FIX
        scene.append({'type': 'flower', 'data': flower_data})
        
    # --- Assemble the final state object ---
    final_state = {
        'bg1': bg1, 'bg2': bg2, 'gradient': gradient, 'palette': palette, 'scene': scene
    }
    
    return final_state


if __name__ == "__main__":
    # --- CONFIGURATION ---
    # Set how many unique scene files you want to generate.
    NUMBER_OF_SCENES_TO_GENERATE = 4

    print(f"Starting batch generation of {NUMBER_OF_SCENES_TO_GENERATE} scenes...")

    batch_timestamp = datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")

    for i in range(1, NUMBER_OF_SCENES_TO_GENERATE + 1):
        print(f"\nGenerating scene {i} of {NUMBER_OF_SCENES_TO_GENERATE}...")
        
        final_scene_state = generate_my_recipe()
        
        output_data = {
            "history": [final_scene_state],
            "histIndex": 0
        }
        
        output_filename = f"procedural_scene_{batch_timestamp}_{i}.json.gz"
        
        with gzip.open(output_filename, 'wt', encoding='utf-8') as f:
            json.dump(output_data, f, separators=(',', ':'))
            
        print(f"  -> Success! Scene saved to '{output_filename}'")

    print(f"\nBatch generation complete.")

### Procedural Generation Using Predefined Nature Color Palettes

In [ ]:
import json
import gzip
import random
import math
import datetime

# ==============================================================================
# 1. UTILITY & HELPER FUNCTIONS
# ==============================================================================

def hsl_to_hex(h, s, l):
    """Converts HSL color to HEX format."""
    s /= 100
    l /= 100
    a = s * min(l, 1 - l)
    def f(n):
        k = (n + h / 30) % 12
        color = l - a * max(-1, min(k - 3, 9 - k, 1))
        return round(255 * color)
    return f'#{f(0):02x}{f(8):02x}{f(4):02x}'

# Seeded Random Number Generator (for deterministic results)
class Mulberry32:
    def __init__(self, seed):
        self.seed = seed
    def __call__(self):
        self.seed = (self.seed + 0x6D2B79F5) & 0xFFFFFFFF
        t = self.seed
        t = ((t ^ (t >> 15)) * 0x1B873593) & 0xFFFFFFFF
        t = t ^ (t >> 7)
        t = ((t ^ (t >> 7)) * 0x6C5565A5) & 0xFFFFFFFF
        t = t ^ (t >> 14)
        return (t ^ (t >> 14)) / 4294967296

# ==============================================================================
# 2. PREDEFINED COLOR PALETTES
# The easiest way to change color schemes is to add or edit palettes here.
# ==============================================================================

COLOR_PALETTES = {
    "Autumn Forest": ["#4f3b2e", "#a66321", "#e39d4b", "#f2d399", "#a3a68b", "#617366", "#3e4a3e", "#2c302d", "#8c5a3c", "#ba8a65"],
    "Oceanic": ["#032b43", "#075a80", "#189ad3", "#66c2f0", "#bfe6f9", "#e0f7ff", "#2a3b47", "#5e7282", "#a7adba", "#dce1e6"],
    "Desert Sunset": ["#2e0c1f", "#5a1839", "#8f254b", "#d8504d", "#f08f65", "#f9d29c", "#c2a38f", "#8c7a70", "#595151", "#332d31"],
    "Winter Dawn": ["#1a2a3a", "#3b526d", "#6b83a1", "#a4b8d0", "#e1eaf2", "#f0f4f7", "#c4a9a4", "#a18683", "#7d6462", "#4f3e3c"],
    "Spring Meadow": ["#2a4d3e", "#4caf50", "#8bc34a", "#cddc39", "#ffeb3b", "#fffde7", "#a1887f", "#d7ccc8", "#5d4037", "#bcaaa4"],
    "Enchanted Forest": ["#1b2d2a", "#3d5245", "#6a8a79", "#a1c3b4", "#dff0e7", "#8e44ad", "#9b59b6", "#be90d4", "#f1e4f3", "#34495e"]
}

# ==============================================================================
# 3. FRACTAL & OBJECT BUILDER FUNCTIONS
# ==============================================================================

def build_mountain_range(start, end, detail, height, jaggedness):
    """Generates mountain ridgeline points using midpoint displacement."""
    points = [start, end]
    displacement = height
    for _ in range(detail):
        new_points = [points[0]]
        for i in range(len(points) - 1):
            p1 = points[i]; p2 = points[i+1]
            mid_x = (p1['x'] + p2['x']) / 2; mid_y = (p1['y'] + p2['y']) / 2
            offset = (random.random() - 0.5) * displacement
            new_points.append({'x': mid_x, 'y': mid_y + offset})
            new_points.append(p2)
        points = new_points
        displacement *= jaggedness
    return points

def build_tree_segments(params):
    """Recursively generates the segment data for a fractal tree."""
    segments = []; rand = Mulberry32(params.get('rngSeed', int(random.random() * 1e9)))
    len_rand_map = [((rand() - 0.5) * params.get('lenRand', 0)) for _ in range(params['levels'])] if params.get('unifyLenPerLevel') else None
    angle_rand_map = [((rand() - 0.5) * params.get('angleRand', 0) * 0.15) for _ in range(params['levels'])] if params.get('unifyAnglePerLevel') else None
    uniform_jitter = (rand() - 0.5) * params.get('angleRand', 0) * 0.15 if params.get('uniformAngleRand') else None

    def branch(x, y, length, angle, depth, level, parent_idx):
        if depth <= 0 or length < 0.6: return None
        x2 = x + length * math.cos(angle); y2 = y - length * math.sin(angle)
        seg_idx = len(segments)
        segments.append({'level': level, 'len': length, 'baseAng': angle, 'parent': parent_idx, 'children': [], 'x1': x, 'y1': y, 'x2': x2, 'y2': y2})
        len_randomness = len_rand_map[level] if (len_rand_map and level < len(len_rand_map)) else (rand() - 0.5) * params.get('lenRand', 0)
        new_length = length * (params.get('lenScale', 0.68) + len_randomness)
        if uniform_jitter is not None: jitter = uniform_jitter
        elif angle_rand_map and level < len(angle_rand_map): jitter = angle_rand_map[level]
        else: jitter = (rand() - 0.5) * params.get('angleRand', 0) * 0.15
        spread = params.get('angle', 25) * math.pi / 180
        left_idx = branch(x2, y2, new_length, angle - spread + jitter, depth - 1, level + 1, seg_idx)
        right_idx = branch(x2, y2, new_length, angle + spread + jitter, depth - 1, level + 1, seg_idx)
        if left_idx is not None: segments[seg_idx]['children'].append(left_idx)
        if right_idx is not None: segments[seg_idx]['children'].append(right_idx)
        return seg_idx
    branch(params['x'], params['y'], params['baseLen'], math.pi / 2, params['levels'], 0, -1)
    return segments

def build_flower_segments(params):
    """Generates flower segments using an L-System."""
    s = 'F'
    for _ in range(params['iter']): s = s.replace('F', 'F[+F]F[-F]F')
    segments, stack = [], []
    x, y, angle = params['cx'], params['cy'], -math.pi / 2; step = params['step']
    for char in s:
        if char == 'F':
            nx, ny = x + step * math.cos(angle), y + step * math.sin(angle)
            segments.append({'x1': x, 'y1': y, 'x2': nx, 'y2': ny}); x, y = nx, ny
        elif char == '+': angle += params['angle'] * math.pi / 180
        elif char == '-': angle -= params['angle'] * math.pi / 180
        elif char == '[': stack.append({'x': x, 'y': y, 'angle': angle})
        elif char == ']' and stack: state = stack.pop(); x, y, angle = state['x'], state['y'], state['angle']
    return segments

def find_flower_tips(flower_data):
    """Finds endpoints that are not start points and saves them to a 'tips' array."""
    if not flower_data.get('segments'): flower_data['tips'] = []; return
    start_points, end_points = set(), {}
    for seg in flower_data['segments']:
        start_points.add(f"{seg['x1']},{seg['y1']}")
        end_points[f"{seg['x2']},{seg['y2']}"] = {'x': seg['x2'], 'y': seg['y2']}
    flower_data['tips'] = [point for key, point in end_points.items() if key not in start_points]

# ==============================================================================
# 4. SCENE GENERATION RECIPE
# ==============================================================================

def generate_my_recipe(canvas_width=1920, canvas_height=1080):
    scene = []
    
    # --- Palette and Background ---
    palette_name = random.choice(list(COLOR_PALETTES.keys()))
    palette = COLOR_PALETTES[palette_name]
    bg1 = palette[0]; bg2 = palette[1]
    gradient = True
    
    # --- Sun/Moon Object ---
    sun_size = random.uniform(canvas_width / 6, canvas_width / 3)
    scene.append({'type': 'celestial', 'data': {'cx': random.uniform(sun_size, canvas_width - sun_size), 'cy': random.uniform(sun_size * 0.75, canvas_height * 0.4), 'size': sun_size, 'glow': sun_size * random.uniform(0.6, 1.0), 'color': random.choice(['#FFFFFF', '#FFD700', '#FFF4E6']), 'alpha': 1.0}})
    
    # --- Cloud Objects (Grouped into Bands) ---
    for _ in range(random.randint(2, 4)):
        band_y = random.uniform(50, canvas_height * 0.5)
        num_clouds_in_band = random.randint(3, 7)
        band_start_x = random.uniform(-canvas_width * 0.2, canvas_width * 0.8)
        for i in range(num_clouds_in_band):
            cloud_dmax = random.uniform(80, 200)
            cloud_puffs = [{'r': random.uniform(cloud_dmax * 0.2, cloud_dmax) / 2, 'color': hsl_to_hex(random.randint(0, 360), random.randint(5, 20), random.randint(85, 98)), 'offsetX': (random.random() - 0.5) * cloud_dmax, 'offsetY': (random.random() - 0.5) * cloud_dmax * 0.5} for _ in range(random.randint(3, 5))]
            scene.append({'type': 'clouds', 'data': {'cx': band_start_x + i * random.uniform(80, 120), 'cy': band_y + random.uniform(-10, 10), 'circles': cloud_puffs, 'blur': random.uniform(10, 20), 'shadowColor': '#FFFFFF', 'shadowX': random.uniform(-10, 10), 'shadowY': random.uniform(-10, 10), 'alpha': random.uniform(0.7, 0.9)}})
    
    # --- Mountain Objects ---
    mountain_ridges = []
    num_mountains = 4
    for i in range(num_mountains):
        base_y = random.uniform(canvas_height * (0.4 + i * 0.12), canvas_height * (0.45 + i * 0.12))
        angle_rad = math.radians(random.uniform(-10, 10)); y_offset = math.tan(angle_rad) * (canvas_width + 100) / 2
        mtn_start = {'x': -50, 'y': base_y - y_offset}; mtn_end = {'x': canvas_width + 50, 'y': base_y + y_offset}
        mtn_data = {'start': mtn_start, 'end': mtn_end, 'detail': 8, 'height': 200 / (i + 1), 'jaggedness': 0.6, 'isSmooth': i == num_mountains - 1, 'color': palette[i], 'alpha': 1.0, 'hasGradient': False, 'color2': None}
        mtn_data['points'] = build_mountain_range(mtn_start, mtn_end, mtn_data['detail'], mtn_data['height'], mtn_data['jaggedness'])
        scene.append({'type': 'mountain', 'data': mtn_data}); mountain_ridges.append(mtn_data['points'])

    # --- Tree Objects ---
    num_trees = 0; TOTAL_TREE_CAP = 100
    for i, ridge in enumerate(mountain_ridges):
        trees_on_this_ridge = random.randint(15 - i * 3, 30 - i * 5)
        for _ in range(trees_on_this_ridge):
            if num_trees >= TOTAL_TREE_CAP: break
            point = random.choice(ridge)
            tree_params = {
                'x': point['x'],
                'y': point['y'],
                'levels': random.randint(6, 8),
                'baseLen': (40 + i * 15) * random.uniform(0.8, 1.2),
                'lenScale': random.uniform(0.55, 0.85),
                'angle': random.uniform(15, 50),
                'lenRand': 0.2,
                'angleRand': 0.2,
                'baseWidth': (8 + i * 3) * random.uniform(0.8, 1.2),
                'widthScale': 0.7,
                'branchColors': palette,
                'levelAlphas': [1.0] * 10,
                'hasBlossoms': i >= num_mountains - 2 and random.random() > 0.5,
                'blossomColor': '#FFFFFF',
                'blossomSize': 3,
                'rngSeed': int(random.random() * 1e9),
                'randomColor': False,
                'randomColorPerLevel': False,
                'uniformAngleRand': False,
                'unifyLenPerLevel': False,
                'unifyAnglePerLevel': False,
                'hasShadow': False, 'shadowColor': '#000000', 'shadowBlur': 0, 'shadowX': 0, 'shadowY': 0
            }
            tree_data = {**tree_params, 'segments': build_tree_segments(tree_params)}; scene.append({'type': 'tree', 'data': tree_data}); num_trees += 1
            
    # --- Ferns and Flowers on the front-most ridge ---
    front_ridge = mountain_ridges[-1]
    for _ in range(random.randint(20, 50)): # Add Ferns
        point = random.choice(front_ridge); y_offset = random.uniform(0, canvas_height * 0.15)
        scene.append({'type': 'fern', 'data': {'cx': point['x'], 'cy': point['y'] + y_offset, 'size': (canvas_height * 0.01) * random.uniform(0.8, 1.5), 'points': 1500, 'color': random.choice(palette), 'rngSeed': int(random.random() * 1e9), 'alpha': 1.0, 'isSpaceFern': True}})
    for _ in range(random.randint(15, 40)): # Add Flowers
        point = random.choice(front_ridge); y_offset = random.uniform(0, canvas_height * 0.15)
        flower_params = {'cx': point['x'], 'cy': point['y'] + y_offset, 'iter': 3, 'angle': random.uniform(50, 70), 'step': (canvas_height * 0.01) * random.uniform(0.5, 0.8), 'stroke': 2.0, 'alpha': 1.0, 'color': random.choice(palette), 'hasBlossoms': True, 'blossomSize': 2, 'blossomColor': hsl_to_hex(random.randint(0,360), 80, 90)}
        flower_data = {**flower_params, 'segments': build_flower_segments(flower_params)}; find_flower_tips(flower_data); scene.append({'type': 'flower', 'data': flower_data})
        
    return {'bg1': bg1, 'bg2': bg2, 'gradient': gradient, 'palette': palette, 'scene': scene}


if __name__ == "__main__":
    NUMBER_OF_SCENES_TO_GENERATE = 5
    print(f"Starting batch generation of {NUMBER_OF_SCENES_TO_GENERATE} scenes...")
    batch_timestamp = datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")
    for i in range(1, NUMBER_OF_SCENES_TO_GENERATE + 1):
        print(f"\nGenerating scene {i} of {NUMBER_OF_SCENES_TO_GENERATE}...")
        final_scene_state = generate_my_recipe()
        output_data = {"history": [final_scene_state], "histIndex": 0}
        output_filename = f"procedural_scene_{batch_timestamp}_{i}.json.gz"
        with gzip.open(output_filename, 'wt', encoding='utf-8') as f:
            json.dump(output_data, f, indent=2) # Using indent=2 for readability
        print(f"  -> Success! Scene saved to '{output_filename}'")
    print(f"\nBatch generation complete.")